In [1]:
import importlib
import functions
importlib.reload(functions) 
from functions import *

In [2]:
BASE_LINK = 'https://data.who.int/countries'

In [3]:
driver = start_browser()

In [4]:
open_page(driver, BASE_LINK)

In [5]:
countries = get_list_of_countries(driver)

In [6]:
print(f"Total {len(countries)} countries found")

Total 194 countries found


In [31]:
def interpolate_life_and_health_expectancy_data(data_list):
    total_data = next((entry for entry in data_list if entry.get(None) == "Total"), None)
    if not total_data:
        return "No 'Total' category found in the data list."

    year_values = {int(year): float(value) for year, value in total_data.items() if year is not None}
    
    interpolated_data = {}
    start_year = 2000
    end_year = 2024

    for year in range(start_year, end_year + 1):
        if year in year_values:
            interpolated_data[year] = year_values[year]
        else:
            previous_years = [y for y in year_values if y < year]
            next_years = [y for y in year_values if y > year]

            if previous_years and next_years:
                prev_year = max(previous_years)
                next_year = min(next_years)

                slope = (year_values[next_year] - year_values[prev_year]) / (next_year - prev_year)
                interpolated_data[year] = year_values[prev_year] + slope * (year - prev_year)
            elif previous_years:
                last_two_years = sorted(previous_years)[-2:]
                if len(last_two_years) == 2:
                    y1, y2 = last_two_years
                    slope = (year_values[y2] - year_values[y1]) / (y2 - y1)
                    interpolated_data[year] = year_values[y2] + slope * (year - y2)

    return interpolated_data


In [ ]:
def interpolate_groupA_data(data):
    def interpolate_years(year_data):
        year_values = {int(year): float(value.strip('%')) if '%' in value else float(value)
                       for item in year_data for year, value in item.items()}

        interpolated = {}
        for year in range(2000, 2025):
            if year in year_values:
                interpolated[year] = year_values[year]
            else:
                previous_years = [y for y in year_values if y < year]
                next_years = [y for y in year_values if y > year]

                if previous_years and next_years:
                    prev_year = max(previous_years)
                    next_year = min(next_years)
                    slope = (year_values[next_year] - year_values[prev_year]) / (next_year - prev_year)
                    interpolated[year] = year_values[prev_year] + slope * (year - prev_year)
                elif previous_years:
                    last_two_years = sorted(previous_years)[-2:]
                    if len(last_two_years) == 2:
                        y1, y2 = last_two_years
                        slope = (year_values[y2] - year_values[y1]) / (y2 - y1)
                        interpolated[year] = year_values[y2] + slope * (year - y2)
                else:
                    interpolated[year] = year_values[min(next_years)]  

        if '%' in year_data[0].get(str(min(year_values.keys())), ''):
            interpolated = {year: f"{value:.1f}%" for year, value in interpolated.items()}
        else:
            interpolated = {year: f"{value:.1f}" for year, value in interpolated.items()}
        return interpolated

    processed_data = {}
    for indicator, year_data in data.items():
        processed_data[indicator] = interpolate_years(year_data)

    return processed_data

In [7]:
Data = []

print('name ', get_country_name(driver))
print('expenditure ', get_current_health_expenditure(driver).split('(')[0])
print('region', get_who_region(driver))
print('income level',get_world_bank_income_level(driver))
print('pop growth', get_population_growth_rate(driver))
print('pop data \n', get_population_data(driver))
interpolate_life_and_health_expectancy_data(get_life_expectancy_data(driver))
interpolate_life_and_health_expectancy_data(get_health_life_expectancy_data(driver))
interpolate_groupA_data(get_indicator_data_for_group_A(driver,get_group_A_indicators(driver)))


In [7]:
open_page(driver, countries[0])


In [8]:
A = get_group_B_indicators(driver)
A

{'People living with tuberculosis (TB)': 'https://data.who.int/indicators/i/13B4226/C288D13?m49=004',
 'Malaria cases': 'https://data.who.int/indicators/i/B868307/442CEA8?m49=004',
 'Road traffic deaths': 'https://data.who.int/indicators/i/B9D9E6A/D6176E2?m49=004',
 'UHC index score': 'https://data.who.int/indicators/i/3805B1E/9A706FD?m49=004',
 'Births attended by skilled health personnel': 'https://data.who.int/indicators/i/F835E3B/1772666?m49=004',
 'Family planning': 'https://data.who.int/indicators/i/F2772F7/8074BD9?m49=004',
 'DTP3 immunization': 'https://data.who.int/indicators/i/48D7D19/F8E084C?m49=004',
 'MCV2 immunization': 'https://data.who.int/indicators/i/B17F8BC/BB4567B?m49=004',
 'Interventions against NTDs': 'https://data.who.int/indicators/i/95935F3/2D6FBE4?m49=004',
 'Density of doctors': 'https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004',
 'Density of nurses': 'https://data.who.int/indicators/i/B54EB15/5C8435F?m49=004',
 'Density of pharmacists': 'https://da

In [58]:
get_indicator_data_for_group_B(driver,A)

{'People living with tuberculosis (TB)': ['Afghanistan: 185 [117 – 255]'],
 'Malaria cases': ['Afghanistan: 9.06 [6.99 – 11.5]'],
 'Road traffic deaths': ['Afghanistan: 24.1'],
 'UHC index score': ['Afghanistan: 41'],
 'Births attended by skilled health personnel': ['Afghanistan'],
 'Family planning': [],
 'DTP3 immunization': ['Afghanistan: 60%'],
 'MCV2 immunization': ['Afghanistan: 42%'],
 'Interventions against NTDs': ['Afghanistan: 14,367,281'],
 'Density of doctors': ['Afghanistan'],
 'Density of nurses': ['Afghanistan'],
 'Density of pharmacists': ['Afghanistan'],
 'Density of dentists': ['Afghanistan'],
 'WASH development assistance': ['Afghanistan: 68']}

In [12]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_transparent_paths_data(driver):
    """
    Retrieves the 'd' attribute of <path> elements with stroke='transparent'.

    Parameters:
    driver: WebDriver instance

    Returns:
    list: A list of 'd' attributes from the found <path> elements, or an empty list if none are found.
    """
    transparent_paths_data = []

    try:
        # Locate all path elements within the SVG
        path_elements = driver.find_elements(By.CSS_SELECTOR, "svg path[stroke='transparent']")

        # Extract the 'd' attribute from each found path
        for path in path_elements:
            d_attribute = path.get_attribute('d')
            transparent_paths_data.append(d_attribute)

    except NoSuchElementException:
        print("No path elements with stroke='transparent' found.")
    
    return transparent_paths_data

# Example usage:
# from selenium import webdriver
# driver = webdriver.Chrome()  # or any other driver you are using
# driver.get("URL_OF_THE_PAGE")
# transparent_paths = get_transparent_paths_data(driver)
# print("Transparent path data:", transparent_paths)


In [10]:
open_page(driver,A['People living with tuberculosis (TB)'])


In [14]:
len(get_transparent_paths_data(driver))

3

## 1


In [ ]:
for country in countries[0:10]:
    print(f"Processing {country}")
    open_page(driver, country)

    name = get_country_name(driver)
    expenditure = get_current_health_expenditure(driver).split('(')[0]
    region_name = get_who_region(driver)
    income_level = get_world_bank_income_level(driver)
    growth_rate = get_population_growth_rate(driver)
    
    population_trend,universal_health_trend=population_data_mapped(driver)
    life_expectency_data = get_life_expectancy_data(driver)
    male_life_expectency,female_life_expectency,over_life_expectency=life_expectency_mapped(driver)
    male_health_expectency,female_health_expectency,overall_health_expectency=health_expectancy_mapped(driver)
    group_A_links = get_group_A_indicators(driver)
    group_B_links = get_group_B_indicators(driver)
    group_A_data = get_indicator_data_for_group_A(driver, group_A_links)
    hiv_data, non_commencial_diseases, suicide_deaths, hypertension_data, adult_obesity, tobaco_use, alcohol_assumption, safely_managed_sanitation, fine_particulate_matter, financial_hardship = process_group_A_data(group_A_data)
    
    group_B_data=get_indicator_data_for_group_B(driver,group_B_links)
    group_B_processed = {}
    for indicator, data_point in group_B_data.items():
        group_B_processed[indicator] = generate_nearby_values(data_point,name)
    indicator_lists = {}
    for indicator in group_B_processed.keys():
        indicator_lists[indicator] = group_B_processed[indicator]
        
    people_tb_values = indicator_lists['People living with tuberculosis (TB)']
    malaria_cases_values = indicator_lists['Malaria cases']
    road_traffic_deaths_values = indicator_lists['Road traffic deaths']
    uhc_index_score_values = indicator_lists['UHC index score']
    births_attended_values = indicator_lists['Births attended by skilled health personnel']
    dTP3_immunization_values = indicator_lists['DTP3 immunization']
    mcv2_immunization_values = indicator_lists['MCV2 immunization']
    interventions_ntds_values = indicator_lists['Interventions against NTDs']
    density_doctors_values = indicator_lists['Density of doctors']
    density_nurses_values = indicator_lists['Density of nurses']
    density_pharmacists_values = indicator_lists['Density of pharmacists']
    density_dentists_values = indicator_lists['Density of dentists']
    wash_development_assistance_values = indicator_lists['WASH development assistance']
    
    for each, universal_pop, male_expectency, female_expectency in zip(
        population_trend, universal_health_trend, male_life_expectency, female_life_expectency
    ):
        new_row = {
        'Country': name,
        'Expenditure': expenditure,
        'Region Name': region_name,
        'Income Level': income_level,
        'Growth Rate': growth_rate,
        'Year': each,
        'Population': population_trend[each],
        
        'Male Life Expectency': male_health_expectency[each],
        'Female Life Expectency': female_health_expectency[each],
        'Hiv Percentage': hiv_data[each],  
        'non_commencial_diseases': non_commencial_diseases[each],
        'suicide_deaths': suicide_deaths[each],
        'hypertension_data': hypertension_data[each],
        'adult_obesity': adult_obesity[each],
        'Tobacco Use': tobaco_use[each],
        'Alcohol Assumption': alcohol_assumption[each],
        'Safely Managed Sanitation': safely_managed_sanitation[each],
        'Fine Particulate Matter': fine_particulate_matter[each],
        'Financial Hardship': financial_hardship[each],
        'People TB': people_tb_values[each],  
        'Malaria Cases': malaria_cases_values[each],
        'Road Traffic Deaths': road_traffic_deaths_values[each],
        'UHC Index Score': uhc_index_score_values[each],
        'Births Attended': births_attended_values[each],
        'DTP3 Immunization': dTP3_immunization_values[each],
        'MCV2 Immunization': mcv2_immunization_values[each],
        'Interventions NTDs': interventions_ntds_values[each],
        'Density of Doctors': density_doctors_values[each],
        'Density of Nurses': density_nurses_values[each],
        'Density of Pharmacists': density_pharmacists_values[each],
        'Density of Dentists': density_dentists_values[each],
        'WASH Development Assistance': wash_development_assistance_values[each]
        }
        data.append(new_row)


In [16]:
rows=[]


In [17]:
for country in countries[0:10]:
    print(f"Processing {country}")
    open_page(driver, country)

    name = get_country_name(driver)
    expenditure = get_current_health_expenditure(driver).split('(')[0]
    region_name = get_who_region(driver)
    income_level = get_world_bank_income_level(driver)
    growth_rate = get_population_growth_rate(driver)
    
    population_trend,universal_health_trend=population_data_mapped(driver)
    life_expectency_data = get_life_expectancy_data(driver)
    male_life_expectency,female_life_expectency,over_life_expectency=life_expectency_mapped(driver)
    male_health_expectency,female_health_expectency,overall_health_expectency=health_expectancy_mapped(driver)
    group_A_links = get_group_A_indicators(driver)
    group_B_links = get_group_B_indicators(driver)
    group_A_data = get_indicator_data_for_group_A(driver, group_A_links)
    hiv_data, non_commencial_diseases, suicide_deaths, hypertension_data, adult_obesity, tobaco_use, alcohol_assumption, safely_managed_sanitation, fine_particulate_matter, financial_hardship = process_group_A_data(group_A_data)
    group_B_data=get_indicator_data_for_group_B(driver,group_B_links)
    group_B_processed = {}
    for indicator, data_point in group_B_data.items():
        group_B_processed[indicator] = generate_nearby_values(data_point,name)
    indicator_lists = {}
    for indicator in group_B_processed.keys():
        indicator_lists[indicator] = group_B_processed[indicator]
    people_tb_values = indicator_lists['People living with tuberculosis (TB)']
    malaria_cases_values = indicator_lists['Malaria cases']
    road_traffic_deaths_values = indicator_lists['Road traffic deaths']
    uhc_index_score_values = indicator_lists['UHC index score']
    births_attended_values = indicator_lists['Births attended by skilled health personnel']
    dTP3_immunization_values = indicator_lists['DTP3 immunization']
    mcv2_immunization_values = indicator_lists['MCV2 immunization']
    interventions_ntds_values = indicator_lists['Interventions against NTDs']
    density_doctors_values = indicator_lists['Density of doctors']
    density_nurses_values = indicator_lists['Density of nurses']
    density_pharmacists_values = indicator_lists['Density of pharmacists']
    density_dentists_values = indicator_lists['Density of dentists']
    wash_development_assistance_values = indicator_lists['WASH development assistance']
    
    for each, universal_pop, male_expectency, female_expectency in zip(
        population_trend, universal_health_trend, male_life_expectency, female_life_expectency
    ):
        new_row = {
        'Country': name,
        'Expenditure': expenditure,
        'Region Name': region_name,
        'Income Level': income_level,
        'Growth Rate': growth_rate,
        
        'Year': each,
        'Population': population_trend[each],
        'Universal Health Trend': universal_health_trend[each],
        'Male Life Expectency': male_health_expectency[each],
        'Female Life Expectency': female_health_expectency[each],
        'Hiv Percentage': hiv_data[each],  
        'non_commencial_diseases': non_commencial_diseases[each],
        'suicide_deaths': suicide_deaths[each],
        'hypertension_data': hypertension_data[each],
        'adult_obesity': adult_obesity[each],
        'Tobacco Use': tobaco_use[each],
        'Alcohol Assumption': alcohol_assumption[each],
        'Safely Managed Sanitation': safely_managed_sanitation[each],
        'Fine Particulate Matter': fine_particulate_matter[each],
        'Financial Hardship': financial_hardship[each],
        'People TB': people_tb_values[each],  
        'Malaria Cases': malaria_cases_values[each],
        'Road Traffic Deaths': road_traffic_deaths_values[each],
        'UHC Index Score': uhc_index_score_values[each],
        'Births Attended': births_attended_values[each],
        'DTP3 Immunization': dTP3_immunization_values[each],
        'MCV2 Immunization': mcv2_immunization_values[each],
        'Interventions NTDs': interventions_ntds_values[each],
        'Density of Doctors': density_doctors_values[each],
        'Density of Nurses': density_nurses_values[each],
        'Density of Pharmacists': density_pharmacists_values[each],
        'Density of Dentists': density_dentists_values[each],
        'WASH Development Assistance': wash_development_assistance_values[each]
        }
        rows.append(new_row)


Processing https://data.who.int/countries/004
Error fetching data for 2006: Message: stale element reference: stale element not found in the current frame
  (Session info: MicrosoftEdge=126.0.2592.102); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5568ca2a167e <unknown>
#1 0x5568c9f5e682 <unknown>
#2 0x5568c9f63b13 <unknown>
#3 0x5568c9f6550d <unknown>
#4 0x5568c9f655b7 <unknown>
#5 0x5568c9fa4973 <unknown>
#6 0x5568c9fcc972 <unknown>
#7 0x5568c9f9f681 <unknown>
#8 0x5568c9fccb3e <unknown>
#9 0x5568c9fea58e <unknown>
#10 0x5568c9fcc713 <unknown>
#11 0x5568c9f9da43 <unknown>
#12 0x5568c9f9e7fe <unknown>
#13 0x5568ca271151 <unknown>
#14 0x5568ca2742ad <unknown>
#15 0x5568ca273d32 <unknown>
#16 0x5568ca274735 <unknown>
#17 0x5568ca2624db <unknown>
#18 0x5568ca274ad1 <unknown>
#19 0x5568ca24ee05 <unknown>
#20 0x5568ca2925b8 <unknown>
#21 0x5568ca292749 <unknown>
#22 

IndexError: list index out of range

In [10]:
open_page(driver, countries[0])

[]
